# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

In [1]:
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 20 

## デレステセリフのスクレイピング

In [3]:
def get_quotes_sute(q):
    """
    デレステで公開されている各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [4]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    モバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != (n - 1):
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [5]:
# deresute id
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262

In [6]:
aa = get_quotes_sute(112)

In [7]:
bb = get_quotes_moba(query="緒方智絵里", debug=True)

crawling page: 1/2
crawling page: 2/2


In [8]:
aa.head(5)

,0
0,アイドルマスター シンデレラガールズ スターライトステージ!
1,お、おはようございます。アーリーモーニングティー、いかがですか…？
2,お昼だけど…もうちょっとだけ…午後のレッスンの予習をしなくっちゃ…
3,ふぁぁ…こんな時間までお疲れさまです。ホットミルク、飲みますか…？
4,プロデューサーさん…もしお暇なら、私…ステージに、立ちたい…です


In [9]:
bb.head(10)

,0,1,2,3
0,緒方智絵里,緒方智絵里,アイドルコメント,緒方…緒方智絵里……です。あの……その……えぇと…が、がんばります……ので……見捨てないでく...
1,緒方智絵里,緒方智絵里,マイスタジオ,あ……おはようございます
2,緒方智絵里,緒方智絵里,マイスタジオ,あの…あ、なんでもないです…
3,緒方智絵里,緒方智絵里,マイスタジオ,(いい天気…お散歩したいな…)
4,緒方智絵里,緒方智絵里,マイスタジオ,○○プロデューサー、あの……見捨てないでくださいね…?
5,緒方智絵里,緒方智絵里,お仕事,がんばり…ます
6,緒方智絵里,緒方智絵里,お仕事,は、はいっ、お仕事ですね
7,緒方智絵里,緒方智絵里,お仕事,わたし、できるかな…
8,緒方智絵里,緒方智絵里,お仕事,○○プロデューサー……い、いなくならないでくださいね?
9,緒方智絵里,緒方智絵里,お仕事（親愛度UP）,○○プロデューサー、わたし……がんばれました…


## モバ，ステの結合と正規化

In [10]:
bb[3] = bb[3].str.replace("○○", "PRODUCER") # OO (プロデューサー名)部分を 置換して統一
# aa + bb のコーパスを作る
docs = pd.Series()
docs = pd.concat([aa[0], bb[3]])
docs = docs.str.replace("プロデューサー", "PRODUCER")
docs = docs.str.replace("[Producer]", "PRODUCER", regex=False)
docs = docs.reset_index(drop=True)

## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)
- [MecabをPythonで使うまで](https://qiita.com/Sak1361/items/47e9ec464ccc770cd65c)

---

$ mecab -d /usr/lib/mecab/dic/mecab-ipadic-neologd

In [11]:
def extractNoun(text,
                mecab_mode = r'-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd',
                noun="名詞"):
    """
    [arg]: sentence
    [ret]: a list of noun words
    """
    tagger = MeCab.Tagger(mecab_mode)
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        surface = node.surface # 表層形を表示(str)
        meta = node.feature.split(",") # ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']
        if meta[0] == noun:
            keywords.append(node.surface)
        node = node.next # 次のノードのアドレスを指定
    return keywords

def tokenize(text, wakati=True, dic="neologd"):
    """
    テキストを形態素解析して分かち書きする
    [arg]
    text: str
    [ret]
    term: str
    """
    import MeCab
    if dic == "neologd":
        mecab_mode = MeCab.Tagger(r'-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd') # debugging
    else:
        mecab_mode = MeCab.Tagger(r'-Owakati')
    if wakati == True: # 分かち書きしたものをそのまま取り出す
        term = mecab_mode.parse(text).strip()
    else: # 品詞で
        term = extractNoun(text, mecab_mode)
    return term

In [12]:
import MeCab
text = "テキストを形態素解析して分かち書きする"
print(extractNoun(text))

['テキスト', '形態素解析', '分かち書き']


In [13]:
print(f'ipa-dic    ： {tokenize(docs[99], dic = "ipa")}')
print(f'neologd-dic： {tokenize(docs[99])}')

ipa-dic    ： あっ 、 PRODUCER さん 、 今日 の わたし の お 仕事 、 どう でし た か ？
neologd-dic： あっ 、 PRODUCER さん 、 今日 の わたし の お仕事 、 どう でし た か ？


In [14]:
from gensim import corpora, models

In [15]:
docs_list = [tokenize(row).split() for row in docs]
dic = corpora.Dictionary(docs_list)
dic.save_as_text('./data/text.dict')  # 保存
#gensim.corpora.Dictionary.load_from_text('./data/text.dict')  # 次回からファイルロード可

In [16]:
print(len(dic)) # dictionaryに登録された単語(形態素)数

1805


In [17]:
# 単語出現頻度を数える
# 余談: bashだと $ sort -t$'\t' -k3 -nr text.dict で単語とその出現頻度回数カウントできる
from collections import defaultdict
freq = defaultdict(int)

for text in docs_list:
    for token in text:
        freq[token] += 1
print(sorted(freq.items(), key=lambda x:x[1], reverse=True))

[('…', 1042), ('、', 891), ('て', 441), ('の', 360), ('。', 355), ('です', 290), ('に', 289), ('さん', 259), ('PRODUCER', 221), ('が', 208), ('た', 203), ('で', 189), ('も', 181), ('ます', 177), ('は', 176), ('な', 155), ('を', 151), ('し', 145), ('ん', 141), ('と', 139), ('!', 135), ('か', 135), ('私', 128), ('？', 113), ('ね', 111), ('から', 100), ('ない', 79), ('って', 78), ('へ', 77), ('い', 61), ('よ', 59), ('…。', 57), ('まし', 56), ('♪', 55), ('だ', 49), ('みんな', 48), ('あの', 47), ('幸せ', 47), ('こと', 45), ('お', 44), ('え', 44), ('でも', 44), ('あ', 44), ('たい', 43), ('アイドル', 39), ('てる', 39), ('すっ', 38), ('見', 36), ('ください', 36), ('くれ', 36), ('けど', 34), ('そう', 33), ('いい', 33), ('なっ', 33), ('ませ', 32), ('っ', 32), ('たら', 31), ('あっ', 31), ('「', 31), ('う', 30), ('みたい', 28), ('その', 27), ('き', 26), ('なぁ', 25), ('わたし', 25), ('この', 24), ('ま', 23), ('クローバー', 23), ('いる', 22), ('よう', 22), ('なく', 22), ('ちゃ', 21), ('なんて', 21), ('お仕事', 21), ('ちょっと', 20), ('?', 20), ('だけ', 19), ('気', 19), ('たくさん', 19), ('ござい', 18), ('でき', 18), ('今', 18), ('気

In [18]:
corpus = [dic.doc2bow(doc) for doc in docs_list] # コーパスの作成
corpora.MmCorpus.serialize('./data/text.mm', corpus)  # 保存

In [19]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 折角計算したのでpickleに保存
import pickle
with open('./data/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

# 次回からはロードできます
# with open('./data/corpus_tfidf.dump', mode='rb') as f:
#     corpus_tfidf = pickle.load(f)

In [24]:
tfidf.idfs.

AttributeError: 'dict' object has no attribute 'head'

In [21]:
# 文書量にもよりますが、数時間程度かかったりもします。
# '18/12/03追記: LdaMulticore で worker の数を増やせばかなり早くなるかもです
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dic,
                             num_topics=50, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)
lda.save('./data/lda.model')  # 保存
#lda = gensim.models.LdaModel.load('./data/lda.model')  # 次回からロード可

In [22]:
lda.print_topic(10)

'0.023*"へ" + 0.015*"ちゃっ" + 0.015*"サイン" + 0.014*"サクランボ" + 0.010*"のに" + 0.010*"唯" + 0.008*"ボケとツッコミ" + 0.008*"普通" + 0.008*"入れる" + 0.008*"なるほど"'

In [23]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)))

tpc_0: 0.041*"なんだか" + 0.019*"言葉" + 0.015*"支え" + 0.014*"声援" + 0.014*"その" + 0.013*"そば" + 0.011*"なっ" + 0.009*"でも" + 0.009*"いる" + 0.009*"たくさん"
tpc_1: 0.018*"うれしい" + 0.014*"たい" + 0.014*"がんばら" + 0.014*"やっ" + 0.014*"だろ" + 0.014*"ふわふわ" + 0.012*"〇〇" + 0.012*"四つ葉のクローバー" + 0.012*"揃い" + 0.011*"がんばっ"
tpc_2: 0.026*"あげ" + 0.020*"これ" + 0.019*"夢" + 0.019*"力" + 0.017*"たい" + 0.015*"お" + 0.014*"られる" + 0.013*"恥ずかしい" + 0.012*"いえ" + 0.012*"席"
tpc_3: 0.052*"がんばり" + 0.023*"いつか" + 0.017*"ツッコミ" + 0.016*"ます" + 0.011*"ぅ" + 0.011*"なり" + 0.009*"ふ" + 0.007*"以外" + 0.007*"綴じ込ん" + 0.007*"しおり"
tpc_4: 0.021*"なら" + 0.017*"す～" + 0.016*"頬" + 0.010*"読み" + 0.010*"描い" + 0.009*"クローバー" + 0.009*"励まさ" + 0.009*"どうぞ" + 0.009*"たい" + 0.008*"夢み"
tpc_5: 0.047*"へ" + 0.040*"すっ" + 0.039*"ま" + 0.034*"みたい" + 0.025*"お仕事" + 0.024*"はいっ" + 0.021*"え" + 0.020*"!" + 0.018*"とっても" + 0.017*"は"
tpc_6: 0.020*"知ら" + 0.019*"プロデュース" + 0.017*"怖く" + 0.015*"世界" + 0.014*"いう" + 0.013*"寒い" + 0.013*"記念" + 0.012*"なら" + 0.012*"一緒" + 0.011*"ない"
tpc_7: 0.016*"終わっ" + 0